## Gradient Boosted Decision Tree

Impleement a GBDT with TensorFlow using the Boston Housing Value dataset as training samples. Can use it for classification (2 classes: value > 23000 dollars or not) and regression (home value as target).

### Boston Housing Dataset
https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

The dataset contains information collected by the U.S Census Service concerning housing in the area of Boston Mass. It was obtained from the StatLib archive (http://lib.stat.cmu.edu/datasets/boston), and has been used extensively throughout the literature to benchmark algorithms. However, these comparisons were primarily done outside of Delve and are thus somewhat suspect. The dataset is small in size with only 506 cases.

In [1]:
from __future__ import print_function

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "1"

import tensorflow as tf
import numpy as np
import copy

In [3]:
# Dataset parameters
num_classes = 2 # 2 classes; above and below 23000 dollars
num_features = 13 # data feature size

# training params
max_steps = 2000
batch_size = 256
learning_rate = 1.0
l1_regul = 0.0
l2_regul = 0.1

#GBDT params
num_batches_per_layer = 1000
num_trees = 10
max_depth = 4


In [4]:
# prepare Boston Housing Dataset
from tensorflow.keras.datasets import boston_housing
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

# for classification we build 2 classes as explained earlier
def to_binary_class(y):
    for i, label in enumerate(y):
        if label >- 23.0:
            y[i] = 1
        else:
            y[i] = 0
    return y

y_train_binary = to_binary_class(copy.deepcopy(y_train))
y_test_binary = to_binary_class(copy.deepcopy(y_test))

65536/57026 [==================================] - 0s 0s/step


## GBDT Classifier

In [5]:
# build an input function

train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={'x': x_train}, y=y_train_binary,
    batch_size=batch_size, num_epochs=None, shuffle=True)
test_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={'x': x_test}, y=y_test_binary,
    batch_size=batch_size, num_epochs=1, shuffle=False)
test_train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x={'x': x_train}, y=y_train_binary,
    batch_size=batch_size, num_epochs=1, shuffle=False)
# GBDT Models from TF Estimator requires 'feature_column' data format.
feature_columns = [tf.feature_column.numeric_column(key='x', shape=(num_features,))]



In [6]:
gbdt_classifier = tf.estimator.BoostedTreesClassifier(
    n_batches_per_layer=num_batches_per_layer,
    feature_columns=feature_columns, 
    n_classes=num_classes,
    learning_rate=learning_rate, 
    n_trees=num_trees,
    max_depth=max_depth,
    l1_regularization=l1_regul, 
    l2_regularization=l2_regul
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\SUJIT\\AppData\\Local\\Temp\\tmphiw_jfrg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
gbdt_classifier.train(train_input_fn, max_steps=max_steps)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'nam

In [8]:

gbdt_classifier.evaluate(test_train_input_fn)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
The value of AUC returned by this may race with the update so this is deprecated. Please use tf.keras.metrics.AUC instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-12-03T16:11:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\SUJIT\AppData\Local\Temp\tmphiw_jfrg\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.31976s
INFO:tensorflow:Finished evaluation at 2021-12-03-16:11:23
INFO:tensorflow:Saving dict for global step 2000: accuracy = 1.0, accuracy_baseline = 1.0, auc = 0.0, auc_precision_recall = 1.0, average_loss = 0.10670168, global_step = 2000, label/mean = 1.0, loss = 0.10670169, precision = 1.0, prediction/mean = 0.89879405, recall = 1.0
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and prob

{'accuracy': 1.0,
 'accuracy_baseline': 1.0,
 'auc': 0.0,
 'auc_precision_recall': 1.0,
 'average_loss': 0.10670168,
 'label/mean': 1.0,
 'loss': 0.10670169,
 'precision': 1.0,
 'prediction/mean': 0.89879405,
 'recall': 1.0,
 'global_step': 2000}

In [9]:
gbdt_classifier.evaluate(test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-12-03T16:11:38
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\SUJIT\AppData\Local\Temp\tmphiw_jfrg\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.28824s
INFO:tensorflow:Finished evaluation at 2021-12-03-16:11:39
INFO:tensorflow:Saving dict for global step 2000: accuracy = 1.0, accuracy_baseline = 1.0, auc = 0.0, auc_precision_recall = 1.0, average_loss = 0.1067017, global_step = 2000, label/mean = 1.0, loss = 0.1067017, precision = 1.0, prediction/mean = 0.8987938, recall = 1.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: C:\Users\SUJIT\AppData\Local\Temp\tmphiw_jfrg\model.ckpt-2000


{'accuracy': 1.0,
 'accuracy_baseline': 1.0,
 'auc': 0.0,
 'auc_precision_recall': 1.0,
 'average_loss': 0.1067017,
 'label/mean': 1.0,
 'loss': 0.1067017,
 'precision': 1.0,
 'prediction/mean': 0.8987938,
 'recall': 1.0,
 'global_step': 2000}